In [35]:
import cv2
import numpy as np
import glob

#check version
print("OpenCv version: {}", cv2.__version__)

OpenCv version: {} 4.5.3


In [36]:
# Set file for calibration
# img_raw: Multiple image files for calibrationm, therefore a "*" is applied
# img_target: The image which the calibration results will be applied
img_raw = './image/checker_*.jpg'
img_target = './image/cam3.jpg'
result_path = './image/cam3_undistored.jpg'
#img_raw = './image/checker_*.jpg'
#img_target = './image/checker_00.jpg'

In [37]:
# Set checker board parameters. A standard opencv 7*10 checker has 6*9 corners
CHECKERBOARD = (6,9)

#Image out factor
fx_resize = 0.3
fy_resize = 0.3

#OC indicator parameters
oc_radius = 10
oc_color = (0, 240, 240)
bg_color = (80, 80, 80)
text_color = (0, 240, 240)
bg_x = 20
bg_y = 0
bg_w = 600
bg_h = 90
text_x = 20
text_y = 20
text_size = 2
text_line_w = 3

In [38]:
# Create arrays and config flags for calibration
_img_shape = None

subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC + cv2.fisheye.CALIB_CHECK_COND + cv2.fisheye.CALIB_FIX_SKEW

objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

In [39]:
# Read image files, convert to gray level, and find corners in images
images = glob.glob(img_raw)
for fname in images:
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
        
N_OK = len(objpoints)
print("Found " + str(N_OK) + " valid images for calibration")

Found 8 valid images for calibration


In [40]:
# Find K and D matrix
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )

print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")
print("OC=(" + str(K[0,2]) + ", " + str(K[1,2]) + ")")

DIM=(2592, 1944)
K=np.array([[1238.8973384191427, 0.0, 1296.7584921501195], [0.0, 1239.1028827686082, 976.3825196094416], [0.0, 0.0, 1.0]])
D=np.array([[-0.1307433594261936], [0.04581714762639465], [-0.13650221908717208], [0.09875685944472529]])
OC=(1296.7584921501195, 976.3825196094416)


In [41]:
DIM=(_img_shape[1], _img_shape[0])

#To display full image without cropping
# Method 1
#nk = K.copy()
#nk[0,0]=K[0,0]/2
#nk[1,1]=K[1,1]/2
#Method 2
new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, DIM, np.eye(3), balance=1)

#Apply opencv fisheye undistort
img = cv2.imread(img_target)
h,w = img.shape[:2]
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), new_K, DIM, cv2.CV_16SC2)
undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

#print(img.shape)
#print(undistorted_img.shape)

In [42]:
# Write undistored image to file
#cv2.imwrite("fisheye_unwarp_result_00.png", undistorted_img)
cv2.imwrite(result_path, undistorted_img)

True

In [43]:
#Draw optical center background and text
cx = int(K[0,2])
cy = int(K[1,2])
text = "(" + "{:04.2f}".format(K[0,2]) + ", " + "{:04.2f}".format(K[1,2]) + ")"

img=cv2.circle(img, (cx, cy), oc_radius, oc_color, -1)
img=cv2.rectangle(img, (cx+bg_x, cy+bg_y), (cx+bg_w, cy-bg_h), bg_color, -1)
img=cv2.putText(img, text, (cx+text_x, cy-text_y), cv2.FONT_HERSHEY_SIMPLEX, text_size, text_color, text_line_w, cv2.LINE_AA)

In [44]:
# Display original image and result
img = cv2.resize(img, None, fx=fx_resize, fy=fy_resize, interpolation=cv2.INTER_AREA)
undistorted_img = cv2.resize(undistorted_img, None, fx=fx_resize, fy=fy_resize, interpolation=cv2.INTER_AREA)
cv2.imshow("original", img)
cv2.imshow("undistorted", undistorted_img)

In [45]:
cv2.waitKey(0)
cv2.destroyAllWindows()